In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [3]:
import sys
sys.path.append('../')

In [4]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline

from utils.processing import *
from task2_recommender import * 


In [5]:
pd.set_option('display.max_columns', 100) 

## Load and Preprocess Data

In [6]:
def drop_cols_for_q2(df):
    drop_columns = [
        'address',
        'title',
        'listing_id',
        'property_name',
        'total_num_units',
        'available_unit_types',
        'property_details_url',
        'elevation',
        'tenure',
        'property_type',
        'floor_level',
        'furnishing',
        'built_year',
        'subzone', 
        'planning_area', 
        'region'
    ]
    return df.drop(columns=drop_columns)

In [7]:
df = pd.read_csv('../data/train.csv')
df = preprocess(df)

adfs = read_aux_csv('../data')
df = join_aux(df, adfs)

df_with_listing_info = df.copy()
df = drop_cols_for_q2(df)
df.head(1)

,num_beds,num_baths,size_sqft,lat,lng,price,property_type_private,property_type_public,tenure_high_year,tenure_low_year,floor_level_ground,floor_level_high,floor_level_low,floor_level_mid,floor_level_penthouse,floor_level_top,furnishing_partial,furnishing_unfurnished,furnishing_unspecified,nearest_mrt_distance_in_km,line_cc,line_ce,line_cg,line_dt,line_ew,line_ne,line_ns,line_te,nearest_pri_sch_distance_in_km,nearest_gep_pri_sch_distance_in_km,gep_pri_sch_within_1km,gep_pri_sch_within_1km_2km,gep_pri_sch_outside_2km,pri_sch_within_500m,pri_sch_outside_500m,nearest_com_centre_distance_in_km,cc_type_BN,cc_type_CR,cc_type_IEBP,cc_type_IEPB,cc_type_IHL,nearest_mall_distance_in_km,area_size,population,density
0,3.0,2.0,1184,1.2819,103.825948,1081500.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.49163,0,0,0,0,1,0,0,0,0.261718,3.344506,0,0,1,1,0,2.47968,1,0,0,0,0,0.519278,0.3508,15120,43101.482326


In [8]:
# Standardisation of all dimensions
pipe = Pipeline([('scaler', StandardScaler())])
X_transformed = pipe.fit_transform(df)

## Top k Recommendations

In Task 2, we fit a K Nearest Neighbour model to the dataset to retrieve listings that are similar to the input listing, and output them as recommendations. There are mainly 3 variations of the model in this notebook. Before we move on to explore the different variations of the model, feel free to refer to the task2_recommender.py file to look at the model class and the input parameters needed for better understanding. 

### 1. Default Setting

In the default setting, features that are continous/encoded are used in the K Nearest Neighbour model. There are 2 parameters to explore in this default setting as shown below.

In [9]:
# Parameters
row_idx = 0 # row index of listing user wants to find similar recommendations to
num_recommendations = 5 # number of recommendations user wants to receive


In [10]:
model1 = knn(df_with_listing_info, X_transformed, pipe)
default_reco = model1.get_top_recommendations(df.iloc[[row_idx]], k=num_recommendations)
default_reco

,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,floor_level,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price,property_type_private,property_type_public,tenure_high_year,tenure_low_year,floor_level_ground,floor_level_high,floor_level_low,floor_level_mid,floor_level_penthouse,floor_level_top,furnishing_partial,furnishing_unfurnished,furnishing_unspecified,nearest_mrt_distance_in_km,line_cc,line_ce,line_cg,line_dt,line_ew,line_ne,line_ns,line_te,nearest_pri_sch_distance_in_km,nearest_gep_pri_sch_distance_in_km,gep_pri_sch_within_1km,gep_pri_sch_within_1km_2km,gep_pri_sch_outside_2km,pri_sch_within_500m,pri_sch_outside_500m,nearest_com_centre_distance_in_km,cc_type_BN,cc_type_CR,cc_type_IEBP,cc_type_IEPB,cc_type_IHL,nearest_mall_distance_in_km,area_size,population,density,region
14895,776886,hdb flat for sale in 118b jalan membina,alexandra / commonwealth (d3),membina 118,property_type_public,tenure_low_year,2003.0,3.0,2.0,1184,NA,unspecified,3 br,NaN,https://www.99.co/singapore/hdb/membina-118-de...,1.281900,103.825948,0,tiong bahru station,bukit merah,1081500.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.491630,0,0,0,0,1,0,0,0,0.261718,3.344506,0,0,1,1,0,2.479680,1,0,0,0,0,0.519278,0.3508,15120,43101.482326,c
4094,288001,hdb flat for sale in 26d jalan membina,alexandra / commonwealth (d3),membina court,property_type_public,tenure_low_year,2009.0,3.0,2.0,969,NA,unspecified,"1, 2, 3, 4, 5 br",NaN,https://www.99.co/singapore/hdb/membina-court-...,1.282239,103.825114,0,tiong bahru station,bukit merah,932400.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.487173,0,0,0,0,1,0,0,0,0.248094,3.406006,0,0,1,1,0,2.380521,1,0,0,0,0,0.520372,0.3508,15120,43101.482326,c
19103,970335,hdb flat for sale in 119d kim tian road,alexandra / commonwealth (d3),kim tian 119,property_type_public,tenure_low_year,2002.0,3.0,2.0,1184,NA,unspecified,3 br,NaN,https://www.99.co/singapore/hdb/kim-tian-119-d...,1.281975,103.827937,0,tiong bahru station,bukit merah,1013200.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.480625,0,0,0,0,1,0,0,0,0.326326,3.151773,0,0,1,1,0,2.692371,1,0,0,0,0,0.492161,0.3508,15120,43101.482326,c
3500,264414,hdb flat for sale in 119d kim tian road,alexandra / commonwealth (d3),kim tian 119,property_type_public,tenure_low_year,2002.0,3.0,2.0,1184,NA,unspecified,3 br,NaN,https://www.99.co/singapore/hdb/kim-tian-119-d...,1.281975,103.827937,0,tiong bahru station,bukit merah,974400.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.480625,0,0,0,0,1,0,0,0,0.326326,3.151773,0,0,1,1,0,2.692371,1,0,0,0,0,0.492161,0.3508,15120,43101.482326,c
15107,786786,hdb flat for sale in 119d kim tian road,alexandra / commonwealth (d3),kim tian 119,property_type_public,tenure_low_year,2002.0,3.0,2.0,990,NA,unspecified,3 br,NaN,https://www.99.co/singapore/hdb/kim-tian-119-d...,1.281975,103.827937,0,tiong bahru station,bukit merah,982700.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.480625,0,0,0,0,1,0,0,0,0.326326,3.151773,0,0,1,1,0,2.692371,1,0,0,0,0,0.492161,0.3508,15120,43101.482326,c


In [11]:
# We can try another example with a different listing (e.g. row_idx = 1000). 
# Since we only need to find the nearest neighbours with existing model, we can set refit_model = False.
row_idx = 1000
model1.get_top_recommendations(df.iloc[[row_idx]], k=num_recommendations, refit_model=False)

,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,floor_level,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price,property_type_private,property_type_public,tenure_high_year,tenure_low_year,floor_level_ground,floor_level_high,floor_level_low,floor_level_mid,floor_level_penthouse,floor_level_top,furnishing_partial,furnishing_unfurnished,furnishing_unspecified,nearest_mrt_distance_in_km,line_cc,line_ce,line_cg,line_dt,line_ew,line_ne,line_ns,line_te,nearest_pri_sch_distance_in_km,nearest_gep_pri_sch_distance_in_km,gep_pri_sch_within_1km,gep_pri_sch_within_1km_2km,gep_pri_sch_outside_2km,pri_sch_within_500m,pri_sch_outside_500m,nearest_com_centre_distance_in_km,cc_type_BN,cc_type_CR,cc_type_IEBP,cc_type_IEPB,cc_type_IHL,nearest_mall_distance_in_km,area_size,population,density,region
18499,943201,4 bed condo for sale in the lilium,31 how sun road,the lilium,property_type_private,tenure_high_year,2021.0,4.0,4.0,1292,NA,unspecified,"studio, 2, 3, 4, 5 br",80.0,https://www.99.co/singapore/condos-apartments/...,1.345552,103.881514,0,upper paya lebar,serangoon,2730000.0,1,0,1,0,0,0,0,0,0,0,0,0,1,0.370288,1,0,0,0,0,0,0,0,0.61717,3.166211,0,0,1,0,1,1.205022,1,0,0,0,0,0.949835,0.8986,16920,18829.290007,ne
1515,170990,4 bed condo for sale in the lilium,29 how sun road,the lilium,property_type_private,tenure_high_year,2021.0,4.0,4.0,1291,NA,unspecified,"studio, 2, 3, 4, 5 br",80.0,https://www.99.co/singapore/condos-apartments/...,1.345552,103.881514,0,upper paya lebar,serangoon,2791200.0,1,0,1,0,0,0,0,0,0,0,0,0,1,0.370288,1,0,0,0,0,0,0,0,0.61717,3.166211,0,0,1,0,1,1.205022,1,0,0,0,0,0.949835,0.8986,16920,18829.290007,ne
3495,264218,4 bed condo for sale in the lilium,31 how sun road,the lilium,property_type_private,tenure_high_year,2021.0,4.0,4.0,1291,NA,unspecified,"studio, 2, 3, 4, 5 br",80.0,https://www.99.co/singapore/condos-apartments/...,1.345552,103.881514,0,upper paya lebar,serangoon,2812200.0,1,0,1,0,0,0,0,0,0,0,0,0,1,0.370288,1,0,0,0,0,0,0,0,0.61717,3.166211,0,0,1,0,1,1.205022,1,0,0,0,0,0.949835,0.8986,16920,18829.290007,ne
19022,967076,4 bed condo for sale in the lilium,31 how sun road,the lilium,property_type_private,tenure_high_year,2021.0,4.0,4.0,1291,NA,unspecified,"studio, 2, 3, 4, 5 br",80.0,https://www.99.co/singapore/condos-apartments/...,1.345552,103.881514,0,upper paya lebar,serangoon,2602200.0,1,0,1,0,0,0,0,0,0,0,0,0,1,0.370288,1,0,0,0,0,0,0,0,0.61717,3.166211,0,0,1,0,1,1.205022,1,0,0,0,0,0.949835,0.8986,16920,18829.290007,ne
3465,262948,4 bed condo for sale in the lilium,29 how sun road,the lilium,property_type_private,tenure_high_year,2021.0,4.0,4.0,1915,NA,unspecified,"studio, 2, 3, 4, 5 br",80.0,https://www.99.co/singapore/condos-apartments/...,1.345552,103.881514,0,upper paya lebar,serangoon,3897900.0,1,0,1,0,0,0,0,0,0,0,0,0,1,0.370288,1,0,0,0,0,0,0,0,0.61717,3.166211,0,0,1,0,1,1.205022,1,0,0,0,0,0.949835,0.8986,16920,18829.290007,ne


### 2. User-specified Preferences

Each user has their own definitions as to what properties are considered similar. Some users consider prices or location as the topmost priority when looking for similar properties, others may place more importance on floor level or type of housing (private vs public). Therefore, user is able to received customised recommendations based on features that they consider as most important when deciding if a property is similar to another. User can specify features in user_preferences below and the KNN model is fitted with only these input features

In [12]:
print('Available features that user can specify as their preferences: \n'
      '\n'
      f'{[x for x in df.columns]}')

Available features that user can specify as their preferences: 

['num_beds', 'num_baths', 'size_sqft', 'lat', 'lng', 'price', 'property_type_private', 'property_type_public', 'tenure_high_year', 'tenure_low_year', 'floor_level_ground', 'floor_level_high', 'floor_level_low', 'floor_level_mid', 'floor_level_penthouse', 'floor_level_top', 'furnishing_partial', 'furnishing_unfurnished', 'furnishing_unspecified', 'nearest_mrt_distance_in_km', 'line_cc', 'line_ce', 'line_cg', 'line_dt', 'line_ew', 'line_ne', 'line_ns', 'line_te', 'nearest_pri_sch_distance_in_km', 'nearest_gep_pri_sch_distance_in_km', 'gep_pri_sch_within_1km', 'gep_pri_sch_within_1km_2km', 'gep_pri_sch_outside_2km', 'pri_sch_within_500m', 'pri_sch_outside_500m', 'nearest_com_centre_distance_in_km', 'cc_type_BN', 'cc_type_CR', 'cc_type_IEBP', 'cc_type_IEPB', 'cc_type_IHL', 'nearest_mall_distance_in_km', 'area_size', 'population', 'density']


In [13]:
# Parameters
row_idx = 0
num_recommendations = 5
user_preferences = ['property_type_private', 'property_type_public', 'line_ew', 'size_sqft'] # feature names that user can specify as their preferences

In [14]:
# DO NOT NEED TO DO ANYTHING
# Get feature index based on feature names indicated by user
feature_dict = dict(zip([x for x in df.columns], range(len(df.columns))))
feature_idx = []
for col in user_preferences:
    feature_idx.append(feature_dict[col])

In [15]:
# User specified preferences
model2 = knn(df_with_listing_info, X_transformed, pipe)
user_specific_reco = model2.get_top_recommendations(df.iloc[[row_idx]], k=num_recommendations, feature_idx=feature_idx, refit_model=True)
user_specific_reco[['listing_id', 'title', 'address', 'property_name', 'price'] + user_preferences]

,listing_id,title,address,property_name,price,property_type_private,property_type_public,line_ew,size_sqft
184,108751,hdb flat for sale in 2d upper boon keng road,eunos / geylang / paya lebar (d14),kallang heights,997500.0,0,1,1,1184
511,123995,hdb flat for sale in 673c jurong west street 65,boon lay / jurong / tuas (d22),673c jurong west street 65,588000.0,0,1,1,1184
558,126103,hdb flat for sale in 9 cantonment close,chinatown / tanjong pagar (d2),cantonment towers,1260000.0,0,1,1,1184
32,101347,hdb flat for sale in 119d kim tian road,alexandra / commonwealth (d3),kim tian 119,995400.0,0,1,1,1184
446,121402,hdb flat for sale in 50 strathmore avenue,alexandra / commonwealth (d3),forfar heights,1155000.0,0,1,1,1184


In [16]:
# We can try another example with a different listing (e.g. row_idx = 1000). 
# Since we only need to find the nearest neighbours with existing model, we can set refit_model = False.
row_idx = 1000
model2.get_top_recommendations(df.iloc[[row_idx]], k=num_recommendations, feature_idx=feature_idx, refit_model=False) \
    [['listing_id', 'title', 'address', 'property_name', 'property_type_private', 'price'] + user_preferences]

,listing_id,title,address,property_name,property_type_private,price,property_type_private,property_type_public,line_ew,size_sqft
457,121863,3 bed condo for sale in piccadilly grand / pic...,5 northumberland road,piccadilly grand / piccadilly galleria,1,2520000.0,1,0,0,1356
293,114114,4 bed condo for sale in bartley vue,6 jalan bunga rampai,bartley vue,1,2543100.0,1,0,0,1356
105,105034,3 bed condo for sale in leedon green,32 leedon heights,leedon green,1,4342800.0,1,0,0,1356
274,113156,4 bed condo for sale in the lilium,33 how sun road,the lilium,1,2958200.0,1,0,0,1356
550,125459,3 bed condo for sale in leedon green,30 leedon heights,leedon green,1,4236800.0,1,0,0,1356


### 3. Controlled Randomness

Sometimes users may not always wish to receive recommendations that are too similar to what they have input. These are times where users would like to receive recommendations that are not so boring or expected. In this variation of model, we introduce a controlled randomness element in the recommendations users receive. 

The aim is to recommend a listing that is not completely random and unsuitable to the user's profile/preferences, nor should it be obviously similar to the input listing. Hence, in this variation, if users ask for 5 recommendations, they will recieve some recommendations that are nearest neighbours to the input listing, but also some some listings that are outside of the 5 nearest neighbours but still within, say, 50 nearest neighbours. This controlled randomness element is affected by these 2 parameters: max_k and degree_of_randomisation.

* max_k: Upper limit of number of nearest neighbours of input listing that the user can receive
* degree_of_randomisation: Between 0 to 1. Determines how many of the recommendations will come from further neighbours. The higher the value, the more recommendations come from further neighbours.

Example:
* num_recommendations = 10
* max_k = 50
* degree_of_randomisation = 0.4

User will receive 10 recommendations in total. Out of the 10 recommendations, 4 recommendations (0.4 * 10) will come from random sampling of neighbours that are outside of top 10 but within top 50 (max_k) nearest neighbours. The remaining 6 recommendations will come from the 6 nearest neighbours. 


In [17]:
# Parameters
row_idx = 0
num_recommendations = 5
max_k = 20
degree_of_randomisation = 0.5


In [18]:
# Controlled Randomness
model3 = knn(df_with_listing_info, X_transformed, pipe)
controlled_randomness_reco = model3.get_top_recommendations(df.iloc[[row_idx]], k=num_recommendations, 
                                                            max_k=max_k, degree_of_randomisation=degree_of_randomisation, refit_model=True)
controlled_randomness_reco

,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,floor_level,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price,property_type_private,property_type_public,tenure_high_year,tenure_low_year,floor_level_ground,floor_level_high,floor_level_low,floor_level_mid,floor_level_penthouse,floor_level_top,furnishing_partial,furnishing_unfurnished,furnishing_unspecified,nearest_mrt_distance_in_km,line_cc,line_ce,line_cg,line_dt,line_ew,line_ne,line_ns,line_te,nearest_pri_sch_distance_in_km,nearest_gep_pri_sch_distance_in_km,gep_pri_sch_within_1km,gep_pri_sch_within_1km_2km,gep_pri_sch_outside_2km,pri_sch_within_500m,pri_sch_outside_500m,nearest_com_centre_distance_in_km,cc_type_BN,cc_type_CR,cc_type_IEBP,cc_type_IEPB,cc_type_IHL,nearest_mall_distance_in_km,area_size,population,density,region
14895,776886,hdb flat for sale in 118b jalan membina,alexandra / commonwealth (d3),membina 118,property_type_public,tenure_low_year,2003.0,3.0,2.0,1184,NA,unspecified,3 br,NaN,https://www.99.co/singapore/hdb/membina-118-de...,1.281900,103.825948,0,tiong bahru station,bukit merah,1081500.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.491630,0,0,0,0,1,0,0,0,0.261718,3.344506,0,0,1,1,0,2.479680,1,0,0,0,0,0.519278,0.3508,15120,43101.482326,c
4094,288001,hdb flat for sale in 26d jalan membina,alexandra / commonwealth (d3),membina court,property_type_public,tenure_low_year,2009.0,3.0,2.0,969,NA,unspecified,"1, 2, 3, 4, 5 br",NaN,https://www.99.co/singapore/hdb/membina-court-...,1.282239,103.825114,0,tiong bahru station,bukit merah,932400.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.487173,0,0,0,0,1,0,0,0,0.248094,3.406006,0,0,1,1,0,2.380521,1,0,0,0,0,0.520372,0.3508,15120,43101.482326,c
9591,540024,hdb flat for sale in 26d jalan membina,alexandra / commonwealth (d3),membina court,property_type_public,tenure_low_year,2009.0,2.0,2.0,646,NA,unspecified,"1, 2, 3, 4, 5 br",NaN,https://www.99.co/singapore/hdb/membina-court-...,1.282239,103.825114,0,tiong bahru station,bukit merah,682500.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.487173,0,0,0,0,1,0,0,0,0.248094,3.406006,0,0,1,1,0,2.380521,1,0,0,0,0,0.520372,0.3508,15120,43101.482326,c
2982,239845,hdb flat for sale in 25a jalan membina,alexandra / commonwealth (d3),membina court,property_type_public,tenure_low_year,2008.0,3.0,2.0,969,NA,unspecified,"1, 2, 3, 4, 5 br",NaN,https://www.99.co/singapore/hdb/membina-court-...,1.283496,103.826046,0,tiong bahru station,bukit merah,945000.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.318191,0,0,0,0,1,0,0,0,0.083865,3.248113,0,0,1,1,0,2.452032,1,0,0,0,0,0.349091,0.3508,15120,43101.482326,c
17104,877577,hdb flat for sale in 26a jalan membina,alexandra / commonwealth (d3),membina court,property_type_public,tenure_low_year,2008.0,2.0,2.0,646,NA,unspecified,"1, 2, 3, 4, 5 br",NaN,https://www.99.co/singapore/hdb/membina-court-...,1.282251,103.825695,0,tiong bahru station,bukit merah,774900.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.461868,0,0,0,0,1,0,0,0,0.226389,3.349246,0,0,1,1,0,2.442799,1,0,0,0,0,0.491866,0.3508,15120,43101.482326,c


## Compare Results

In [19]:
# row_idx = 0
df_with_listing_info.iloc[[0]]

,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,floor_level,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price,property_type_private,property_type_public,tenure_high_year,tenure_low_year,floor_level_ground,floor_level_high,floor_level_low,floor_level_mid,floor_level_penthouse,floor_level_top,furnishing_partial,furnishing_unfurnished,furnishing_unspecified,nearest_mrt_distance_in_km,line_cc,line_ce,line_cg,line_dt,line_ew,line_ne,line_ns,line_te,nearest_pri_sch_distance_in_km,nearest_gep_pri_sch_distance_in_km,gep_pri_sch_within_1km,gep_pri_sch_within_1km_2km,gep_pri_sch_outside_2km,pri_sch_within_500m,pri_sch_outside_500m,nearest_com_centre_distance_in_km,cc_type_BN,cc_type_CR,cc_type_IEBP,cc_type_IEPB,cc_type_IHL,nearest_mall_distance_in_km,area_size,population,density,region
0,100043,hdb flat for sale in 118b jalan membina,alexandra / commonwealth (d3),membina 118,property_type_public,tenure_low_year,2003.0,3.0,2.0,1184,NA,unspecified,3 br,NaN,https://www.99.co/singapore/hdb/membina-118-de...,1.2819,103.825948,0,tiong bahru station,bukit merah,1081500.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.49163,0,0,0,0,1,0,0,0,0.261718,3.344506,0,0,1,1,0,2.47968,1,0,0,0,0,0.519278,0.3508,15120,43101.482326,c


In [20]:
# default_reco
default_reco

,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,floor_level,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price,property_type_private,property_type_public,tenure_high_year,tenure_low_year,floor_level_ground,floor_level_high,floor_level_low,floor_level_mid,floor_level_penthouse,floor_level_top,furnishing_partial,furnishing_unfurnished,furnishing_unspecified,nearest_mrt_distance_in_km,line_cc,line_ce,line_cg,line_dt,line_ew,line_ne,line_ns,line_te,nearest_pri_sch_distance_in_km,nearest_gep_pri_sch_distance_in_km,gep_pri_sch_within_1km,gep_pri_sch_within_1km_2km,gep_pri_sch_outside_2km,pri_sch_within_500m,pri_sch_outside_500m,nearest_com_centre_distance_in_km,cc_type_BN,cc_type_CR,cc_type_IEBP,cc_type_IEPB,cc_type_IHL,nearest_mall_distance_in_km,area_size,population,density,region
14895,776886,hdb flat for sale in 118b jalan membina,alexandra / commonwealth (d3),membina 118,property_type_public,tenure_low_year,2003.0,3.0,2.0,1184,NA,unspecified,3 br,NaN,https://www.99.co/singapore/hdb/membina-118-de...,1.281900,103.825948,0,tiong bahru station,bukit merah,1081500.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.491630,0,0,0,0,1,0,0,0,0.261718,3.344506,0,0,1,1,0,2.479680,1,0,0,0,0,0.519278,0.3508,15120,43101.482326,c
4094,288001,hdb flat for sale in 26d jalan membina,alexandra / commonwealth (d3),membina court,property_type_public,tenure_low_year,2009.0,3.0,2.0,969,NA,unspecified,"1, 2, 3, 4, 5 br",NaN,https://www.99.co/singapore/hdb/membina-court-...,1.282239,103.825114,0,tiong bahru station,bukit merah,932400.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.487173,0,0,0,0,1,0,0,0,0.248094,3.406006,0,0,1,1,0,2.380521,1,0,0,0,0,0.520372,0.3508,15120,43101.482326,c
19103,970335,hdb flat for sale in 119d kim tian road,alexandra / commonwealth (d3),kim tian 119,property_type_public,tenure_low_year,2002.0,3.0,2.0,1184,NA,unspecified,3 br,NaN,https://www.99.co/singapore/hdb/kim-tian-119-d...,1.281975,103.827937,0,tiong bahru station,bukit merah,1013200.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.480625,0,0,0,0,1,0,0,0,0.326326,3.151773,0,0,1,1,0,2.692371,1,0,0,0,0,0.492161,0.3508,15120,43101.482326,c
3500,264414,hdb flat for sale in 119d kim tian road,alexandra / commonwealth (d3),kim tian 119,property_type_public,tenure_low_year,2002.0,3.0,2.0,1184,NA,unspecified,3 br,NaN,https://www.99.co/singapore/hdb/kim-tian-119-d...,1.281975,103.827937,0,tiong bahru station,bukit merah,974400.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.480625,0,0,0,0,1,0,0,0,0.326326,3.151773,0,0,1,1,0,2.692371,1,0,0,0,0,0.492161,0.3508,15120,43101.482326,c
15107,786786,hdb flat for sale in 119d kim tian road,alexandra / commonwealth (d3),kim tian 119,property_type_public,tenure_low_year,2002.0,3.0,2.0,990,NA,unspecified,3 br,NaN,https://www.99.co/singapore/hdb/kim-tian-119-d...,1.281975,103.827937,0,tiong bahru station,bukit merah,982700.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.480625,0,0,0,0,1,0,0,0,0.326326,3.151773,0,0,1,1,0,2.692371,1,0,0,0,0,0.492161,0.3508,15120,43101.482326,c


In [21]:
# user_specific_reco
user_specific_reco

,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,floor_level,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price,property_type_private,property_type_public,tenure_high_year,tenure_low_year,floor_level_ground,floor_level_high,floor_level_low,floor_level_mid,floor_level_penthouse,floor_level_top,furnishing_partial,furnishing_unfurnished,furnishing_unspecified,nearest_mrt_distance_in_km,line_cc,line_ce,line_cg,line_dt,line_ew,line_ne,line_ns,line_te,nearest_pri_sch_distance_in_km,nearest_gep_pri_sch_distance_in_km,gep_pri_sch_within_1km,gep_pri_sch_within_1km_2km,gep_pri_sch_outside_2km,pri_sch_within_500m,pri_sch_outside_500m,nearest_com_centre_distance_in_km,cc_type_BN,cc_type_CR,cc_type_IEBP,cc_type_IEPB,cc_type_IHL,nearest_mall_distance_in_km,area_size,population,density,region
184,108751,hdb flat for sale in 2d upper boon keng road,eunos / geylang / paya lebar (d14),kallang heights,property_type_public,tenure_low_year,2006.0,3.0,2.0,1184,NA,unspecified,"2, 3, 4, 5 br",NaN,https://www.99.co/singapore/hdb/kallang-height...,1.313190,103.872341,0,boon keng,kallang,997500.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.216823,0,0,0,0,1,0,0,0,1.297058,2.889250,0,0,1,0,1,2.416385,1,0,0,0,0,0.978161,0.4109,11790,28693.112679,c
511,123995,hdb flat for sale in 673c jurong west street 65,boon lay / jurong / tuas (d22),673c jurong west street 65,property_type_public,NA,2002.0,3.0,2.0,1184,NA,unspecified,NaN,122.0,https://www.99.co/singapore/hdb/673c-jurong-we...,1.344722,103.701277,0,jurong west central,jurong west,588000.0,0,1,0,0,0,0,0,0,0,0,0,0,1,0.863708,0,0,0,0,1,0,0,0,0.255000,9.719596,0,0,1,1,0,1.188109,0,0,1,0,0,0.567819,1.4045,63760,45396.938412,w
558,126103,hdb flat for sale in 9 cantonment close,chinatown / tanjong pagar (d2),cantonment towers,property_type_public,tenure_low_year,2002.0,3.0,2.0,1184,NA,unspecified,"1, 2, 3, 4 br",NaN,https://www.99.co/singapore/hdb/cantonment-tow...,1.275315,103.839205,0,everton park,bukit merah,1260000.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.555886,0,0,0,0,1,1,0,0,0.094100,2.811611,0,0,1,1,0,2.126935,0,1,0,0,0,0.765929,0.6357,7810,12285.669341,c
32,101347,hdb flat for sale in 119d kim tian road,alexandra / commonwealth (d3),kim tian 119,property_type_public,tenure_low_year,2002.0,3.0,2.0,1184,high,unspecified,3 br,NaN,https://www.99.co/singapore/hdb/kim-tian-119-d...,1.281975,103.827937,0,tiong bahru station,bukit merah,995400.0,0,1,0,1,0,1,0,0,0,0,0,0,1,0.480625,0,0,0,0,1,0,0,0,0.326326,3.151773,0,0,1,1,0,2.692371,1,0,0,0,0,0.492161,0.3508,15120,43101.482326,c
446,121402,hdb flat for sale in 50 strathmore avenue,alexandra / commonwealth (d3),forfar heights,property_type_public,tenure_low_year,2006.0,3.0,2.0,1184,NA,unspecified,"studio, 1, 2, 3, 4, 5 br",247.0,https://www.99.co/singapore/hdb/forfar-heights...,1.293847,103.808765,0,margaret drive,queenstown,1155000.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.308878,0,0,0,0,1,0,0,0,0.262659,3.001192,0,0,1,1,0,0.862332,1,0,0,0,0,0.705157,1.0266,16090,15673.095656,c


In [22]:
# controlled_randomness_reco
controlled_randomness_reco

,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,floor_level,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price,property_type_private,property_type_public,tenure_high_year,tenure_low_year,floor_level_ground,floor_level_high,floor_level_low,floor_level_mid,floor_level_penthouse,floor_level_top,furnishing_partial,furnishing_unfurnished,furnishing_unspecified,nearest_mrt_distance_in_km,line_cc,line_ce,line_cg,line_dt,line_ew,line_ne,line_ns,line_te,nearest_pri_sch_distance_in_km,nearest_gep_pri_sch_distance_in_km,gep_pri_sch_within_1km,gep_pri_sch_within_1km_2km,gep_pri_sch_outside_2km,pri_sch_within_500m,pri_sch_outside_500m,nearest_com_centre_distance_in_km,cc_type_BN,cc_type_CR,cc_type_IEBP,cc_type_IEPB,cc_type_IHL,nearest_mall_distance_in_km,area_size,population,density,region
14895,776886,hdb flat for sale in 118b jalan membina,alexandra / commonwealth (d3),membina 118,property_type_public,tenure_low_year,2003.0,3.0,2.0,1184,NA,unspecified,3 br,NaN,https://www.99.co/singapore/hdb/membina-118-de...,1.281900,103.825948,0,tiong bahru station,bukit merah,1081500.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.491630,0,0,0,0,1,0,0,0,0.261718,3.344506,0,0,1,1,0,2.479680,1,0,0,0,0,0.519278,0.3508,15120,43101.482326,c
4094,288001,hdb flat for sale in 26d jalan membina,alexandra / commonwealth (d3),membina court,property_type_public,tenure_low_year,2009.0,3.0,2.0,969,NA,unspecified,"1, 2, 3, 4, 5 br",NaN,https://www.99.co/singapore/hdb/membina-court-...,1.282239,103.825114,0,tiong bahru station,bukit merah,932400.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.487173,0,0,0,0,1,0,0,0,0.248094,3.406006,0,0,1,1,0,2.380521,1,0,0,0,0,0.520372,0.3508,15120,43101.482326,c
9591,540024,hdb flat for sale in 26d jalan membina,alexandra / commonwealth (d3),membina court,property_type_public,tenure_low_year,2009.0,2.0,2.0,646,NA,unspecified,"1, 2, 3, 4, 5 br",NaN,https://www.99.co/singapore/hdb/membina-court-...,1.282239,103.825114,0,tiong bahru station,bukit merah,682500.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.487173,0,0,0,0,1,0,0,0,0.248094,3.406006,0,0,1,1,0,2.380521,1,0,0,0,0,0.520372,0.3508,15120,43101.482326,c
2982,239845,hdb flat for sale in 25a jalan membina,alexandra / commonwealth (d3),membina court,property_type_public,tenure_low_year,2008.0,3.0,2.0,969,NA,unspecified,"1, 2, 3, 4, 5 br",NaN,https://www.99.co/singapore/hdb/membina-court-...,1.283496,103.826046,0,tiong bahru station,bukit merah,945000.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.318191,0,0,0,0,1,0,0,0,0.083865,3.248113,0,0,1,1,0,2.452032,1,0,0,0,0,0.349091,0.3508,15120,43101.482326,c
17104,877577,hdb flat for sale in 26a jalan membina,alexandra / commonwealth (d3),membina court,property_type_public,tenure_low_year,2008.0,2.0,2.0,646,NA,unspecified,"1, 2, 3, 4, 5 br",NaN,https://www.99.co/singapore/hdb/membina-court-...,1.282251,103.825695,0,tiong bahru station,bukit merah,774900.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.461868,0,0,0,0,1,0,0,0,0.226389,3.349246,0,0,1,1,0,2.442799,1,0,0,0,0,0.491866,0.3508,15120,43101.482326,c
